In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
nltk.download("stopwords")
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor 
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor
import os

In [ ]:
# фиксируем random_seed для воспроизводимости

# Kaggle
#DIR_TEST  = '../input/sf-dst-car-price-prediction/' 
#DIR_TRAIN   = '../input/all-auto-ru-09-09-2020/'
#train = pd.read_csv(DIR_TRAIN + 'all_auto_ru_09_09_2020.csv')
#test = pd.read_csv(DIR_TEST + 'test.csv')
#submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

# Local
train = pd.read_csv('all_auto_ru_09_09_2020.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

RANDOM_SEED = 42
NOWDAYS = 2020
VAL_SIZE   = 0.2

In [ ]:
display(train.head(2))
test.head(2)

In [ ]:
print(f'{train.info()}, {test.info()}')

## Сравнение датасетов перед слиянием

In [ ]:
# Количество дупликатов в трейне

train.duplicated().sum()

In [ ]:
# Удалим дупликаты

train.drop_duplicates(inplace=True)

Видим, что в тренировочном и тестовом датасетах разное количество фичей и разные типы данных. Создадим функцию для сравнения датасетов друг с другом

In [ ]:
def check_dfs(df_1, df_2):
    '''
    Данная функция сравнивает между собой два датасета 
    по типам данных признаков и количеству уникальных значений
    '''

    columns_1, columns_2 = list(df_1.columns), list(df_2.columns)
    train_dict, test_dict = {}, {}
    train_dict['train_feats'], test_dict['test_feats'] = columns_1, columns_2
    train_dict['train_types'], test_dict['test_types'] = df_1.dtypes, df_2.dtypes
    train_dict['train_sample'], test_dict['test_sample'] = df_1.loc[10].values, df_2.loc[10].values
    train_dict['nunique_train'], test_dict['nunique_test'] = df_1.nunique().values, df_2.nunique().values

    train_df, test_df = pd.DataFrame.from_dict(train_dict), pd.DataFrame.from_dict(test_dict)
    df_insert = pd.DataFrame(columns=['< - >'])
    check_df = pd.concat([train_df, df_insert, test_df], axis=1)
    check_df.reset_index(inplace=True)
    check_df['< - >'] = '| - |'
    del check_df['index']
    display(check_df)

    temp_dict = {}
    list_1, list_2, list_3, list_4, list_5 = [], [], [], [], []

    for i in range(len(check_df)):
        if str(check_df['train_types'][i]) != str(check_df['test_types'][i]):
            list_1.append(check_df['train_feats'][i])
            list_2.append(check_df['test_feats'][i])
            list_3.append(str(check_df['train_types'][i]) + ' != ' + str(check_df['test_types'][i]))
            list_4.append(i)
        if check_df['nunique_test'][i]>0 and check_df['nunique_train'][i] != check_df['nunique_test'][i]:
            list_5.append(i)
    temp_dict['index'] = list_4
    temp_dict['train_feats'] = list_1
    temp_dict['не совпадают типы'] = list_3
    temp_dict['test_feats'] = list_2
    temp_df = pd.DataFrame.from_dict(temp_dict)
    temp_df.set_index('index', inplace=True)
    print(f'Резюме:\n1. Не совпали типы в:= {len(temp_df)} столбцах\n')
    print(f'2. Уникальные значения различаются в:= {len(list_5)} столбцах {list_5}')
    display(temp_df)

In [ ]:
check_dfs(train, test)

После запуска выявили список столбцов, не полезных в дальнейшей работе.
Это признаки, имеющие единственное значение, признаки не имеющие пары в обучающем и тестовом датасетах, признаки,содержащие консолидированную информацию, уже сохраненную в других признаках.
Некоторые признаки следует рассмотреть отдельно.



In [ ]:
# Признак model в train и model_name в test

print(train.model.value_counts())
print()
print(test.model_name.value_counts())

# Оба данных признака содержат информацию о названии модели авто.
# Переведем все значения в нижний регистр и переименуем столбец в тренировочном датасете в model_name

train.rename(columns={'model':'model_name'}, inplace=True)
train.model_name = train.model_name.str.lower()
test.model_name = test.model_name.str.lower()

print(train.model_name.value_counts())
print()
print(test.model_name.value_counts())

In [ ]:
# Признаки сomplectation_dict, equipment_dict и Комплектация

print(train['Комплектация'].value_counts())
print()
print(test.complectation_dict.value_counts())
print()
print(f'Количество пропусков в столбце Комплектация {train["Комплектация"].isna().sum()}')
print()
print(f'Количество пропусков в столбце complectation_dict {test.complectation_dict.isna().sum()}')
print()

# Подставим вместо пропущенных значений в тестовом признаке заглушку из тренировочного датасета
# Переименуем тренировочный признак в complectation_dict

test.complectation_dict.fillna(train["Комплектация"].value_counts().index[0], inplace=True)
train.rename(columns={'Комплектация':'complectation_dict'}, inplace=True)

print(f'Количество пропусков в столбце complectation_dict после замены NaN {test.complectation_dict.isna().sum()}')
print()
print(train.columns)

In [ ]:
cols_to_drop = ['Состояние', 'Таможня', 'car_url', 'equipment_dict', 'image', 
                'model_info', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'hidden', 'start_date']
                
for col in cols_to_drop:
    if col in list(train.columns):
        train.drop(columns=col, inplace=True)
    if col in list(test.columns):
        test.drop(columns=col, inplace=True)

In [ ]:
# Определим наличие новых дупликатов после удаление колонок

train.duplicated().sum()

In [ ]:
# Удалим дупликаты

train.drop_duplicates(inplace=True)

In [ ]:
check_dfs(train, test)

### Приведение признаков тренировочного к формату из тестового датасета

Признаки с несоответсвием типов данных (modelDate, numberOfDoors, enginePower, Владельцы)

In [ ]:
train.isna().sum()

Вероятно есть строка с одними пропусками, проверим данную теорию

In [ ]:
train.dropna(subset=['modelDate']).isna().sum()

Теория оказалась верна, удаляем данную строку вместе со строками, где не спарсилась цена (наш таргет).

In [ ]:
train.dropna(subset=['modelDate', 'price'], inplace=True)

In [ ]:
# float != int несоответствие

cols_to_int = ['modelDate', 'numberOfDoors']
for col in cols_to_int:
    train[col] = train[col].astype('int')

cols_to_object = ['enginePower', 'Владельцы']
for col in cols_to_object:
    train[col] = train[col].astype('object')

Признак bodyType

In [ ]:
print(train.bodyType.unique()[:10])
print(test.bodyType.unique()[:10])

In [ ]:
# Пока что просто сменим регистр в тренировочном датасете на нижний

train['bodyType'] = train.bodyType.str.lower()

print(train.bodyType.unique()[:10])

Признак *color*

In [ ]:
print(train.color.unique())
print(test.color.unique())

In [ ]:
# В тренировочном сете цвета представлены в HEX формате, переведем их в формат тестового датасета

color_map = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый',
             '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый',
             '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 'FFC0CB':'розовый'}
train['color'] = train.color.map(color_map)

Признак vehicleTransmission

In [ ]:
print(train.vehicleTransmission.unique())
print(test.vehicleTransmission.unique())

In [ ]:
# Просто переведем данные в формат теста

transmission_map = {'MECHANICAL':'механическая', 'AUTOMATIC':'автоматическая', 'ROBOT':'роботизированная', 'VARIATOR':'вариатор'}
train['vehicleTransmission'] = train.vehicleTransmission.map(transmission_map)

print(train.vehicleTransmission.unique())

Признак engineDisplacement

In [ ]:
print(train.engineDisplacement.unique())
print(test.engineDisplacement.unique())

Полный бардак.. попробуем вытащить объем двигателя их столбца name с помощью регулярных выражений

In [ ]:
def displacement_from_name(row):
    row = str(row)
    result = re.findall('\d\.\d', row)
    if result == []:
        return None
    return result[0]

# создадим переменную куда сохраним старый признак "на всякий случай" и перезапишем данный в столбец при помощи нашей функции

train.rename(columns={'engineDisplacement':'engineDisplacement2'}, inplace=True)
engineDisplacementOld = train['engineDisplacement2']
train['engineDisplacement'] = train['name'].apply(displacement_from_name)

In [ ]:
print(train.engineDisplacement.unique())

In [ ]:
# В качестве исключения изменим test значения убрав 'LTR'

test['engineDisplacement'] = test.engineDisplacement.apply(lambda x: np.nan if x == ' LTR' else x.split(' ')[0])

Признак *Руль*

In [ ]:
print(train['Руль'].unique())
print(test['Руль'].unique())

In [ ]:
# Просто переведем данные в формат теста

wheel_map = {'LEFT':'Левый', 'RIGHT':'Правый'}
train['Руль'] = train['Руль'].map(wheel_map)

print(train['Руль'].unique())

Признак *ПТС*

In [ ]:
print(train['ПТС'].unique())
print(test['ПТС'].unique())

In [ ]:
# Просто переведем данные в формат теста

pts_map = {'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'}
train['ПТС'] = train['ПТС'].map(pts_map)

print(train['ПТС'].unique())

Признак *Владельцы*

In [ ]:
print(train['Владельцы'].unique())
print(test['Владельцы'].unique())

In [ ]:
# Просто переведем данные в формат теста

own_map = {3.0:'3 или более', 2.0:'2\xa0владельца', 1.0:'1\xa0владелец'}
train['Владельцы'] = train['Владельцы'].map(own_map)

print(train['Владельцы'].unique())

Признак enginePower

In [ ]:
print(train['enginePower'].unique()[:10])
print(test['enginePower'].unique()[:10])

In [ ]:
# Просто переведем данные в формат теста

train['enginePower'] = train['enginePower'].apply(lambda x: str(int(x))+' N12')

print(train['enginePower'].unique()[:10])

Признаки *fuelType, name, productionDate, vehicleConfiguration, description, mileage, Привод, Владение* по первому взгляду на таблицу не вызвали сомнений. Проверим их отдельно на всякий случай


In [ ]:
print(f'brand train: {train.brand.unique()}\n')
print(f'brand test: {test.brand.unique()}\n')
print()
print(f'fuelType train: {train.fuelType.unique()}\n')
print(f'fuelType test: {test.fuelType.unique()}\n')
print()
print(f'name train: {train.name.unique()}\n')
print(f'name test: {test.name.unique()}\n')
print()
print(f'productionDate train: {train.productionDate.unique()}\n')
print(f'productionDate test: {test.productionDate.unique()}\n')
print()
print(f'vehicleConfiguration train:\n{train.vehicleConfiguration.sample(5)}\n')
print(f'vehicleConfiguration test:\n{test.vehicleConfiguration.sample(5)}\n')
print()
print(f'description train:\n{train.description.sample(5)}\n')
print(f'description test:\n{test.description.sample(5)}\n')
print()
print(f'mileage train:\n{train.mileage.sample(5)}\n')
print(f'mileage test:\n{test.mileage.sample(5)}\n')
print()
print(f'Привод train: {train["Привод"].unique()}\n')
print(f'Привод test: {test["Привод"].unique()}\n')
print()
print(f'Владение train:\n{train["Владение"].sample(5)}\n')
print(f'Владение test:\n{test["Владение"].sample(5)}\n')

Кроме "Владение" все признаки в норме. Признак Владение в train представляет из себя словарь, в то время как в test это просто строковое описание. Пока что оставим до отдельного анализа

Таргет признак *price*

In [ ]:
train.price.hist()

In [ ]:
train.price.describe()

In [ ]:
# из-за большого разбега по стоимости возьмем логарифм от цены

train['price_log'] = np.log(train.price)

In [ ]:
train.price_log.hist()

Финальное сравнение датасетов

In [ ]:
check_dfs(train, test)

Типы данных не совпали лишь в трех признаках:

1. Таргет признак price и его логарифм
2. Созданная нами копия старого признака engineDisplacement

# Слияние и предварительный анализ датасета

In [ ]:
# Создадим признак для разделения датасетов

train['train'] = 1 # Здесь у нас тренировочный датасет
test['train'] = 0  # А тут тестовый

# Объединим наши датасеты

df = train.append(test, sort=False).reset_index(drop=True)

In [ ]:
# Проверим после слияния 

df.sample(3)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

В данный момент не будем расписывать, какой признак к какой категории принадлежит. Сделаем это после отдельного анализа каждого из них и добавления новых фичей.
На первый взгляд по признакам:
1. В признаке bodyType тренировочного датасета слишком много значений, в которых упоминается либо модель, либо количество дверей. Возможно имеет смысл обобщить значения, убрав лишнюю информацию, которая уже присутствует в виде других признаков.
2. Во многих признаках из трениовочного датасета количество значений выше, чем в тестовом. Возможно для улучшения работы модели потребуется обобщать значения, выходящие за рамки тестового датасета (хотя в моем случае это не желательно, так как я пытаюсь сделать именно воспоизводимую модель с разными данными, а не только с теми, что есть в тесте).

# EDA And Feature Engineering

In [ ]:
# Составим функцию для предварительного анализа каждого признака

def col_info(col):
    print('Количество пропусков: {}\n'.format(col.isna().sum()))
    print('{}\n'.format(col.describe()))
    print('Распределение:\n{}\n'.format(col.value_counts()))

In [ ]:
# Создадим списки с типами наших значений, куда будем отправлять каждый из признаков после анализа

cat_cols = []  # Категориальные признаки
num_cols = []  # Числовые признаки
bin_cols = []  # Бинарные признаки
bool_cols = [] # Логические признаки
ordinal_cat_cols = [] # Категориальные порядковые признаки

## 1. bodyType

In [ ]:
col_info(df.bodyType)

In [ ]:
col_info(df[df.train == 1].bodyType)

In [ ]:
col_info(df[df.train == 0].bodyType)

In [ ]:
sns.set()
plt.figure(figsize=(21, 8))
order = df[df.train == 1].groupby('bodyType').price_log.median().sort_values().index

g = sns.boxplot(x = 'bodyType', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / bodyType_train')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

Довольно много специфических типов кузова в тренировочной части датасета. Я вижу два варианта развития событий
1. Объединить все типы в отдельные категории без лишней информации (тип дверей, тип кабины). Например: *седан 2 дв.* станет просто *седан*. Таким образом повысится влияние других признаков в работе модели (модель, кол. дверей).
2. Подогнать признаки из тренировочной части под тестовую, а лишние запихать в отдельное значение *Other*
3. Удалить лишние значения, не входящие в тест, после чего либо оставить все как есть, либо сделать действия из пункта 1

Планирую попробовать все варианты и сравнить качество работы модели

**1 Вариант**

С первого взгляда можно заметить, что общее название кузова у всех значений находится первым словом. Проверим данную теорию

In [ ]:
# train

print(df[df.train == 1].bodyType.apply(lambda x: x.split(' ')[0]).unique())
print(len(df[df.train == 1].bodyType.apply(lambda x: x.split(' ')[0]).unique()))
print()

# test

print(df[df.train == 0].bodyType.apply(lambda x: x.split(' ')[0]).unique())
print(len(df[df.train == 0].bodyType.apply(lambda x: x.split(' ')[0]).unique()))
print()

# Общий

print(df.bodyType.apply(lambda x: x.split(' ')[0]).unique())
print(len(df.bodyType.apply(lambda x: x.split(' ')[0]).unique()))

Никаких лишних значений по типу *2 дв.* мы не получили, а количество уникальных значений в обеих частях одинаково. Отразим данное преобразование на нашем датасете

In [ ]:
df['bodyType_1'] = df.bodyType.apply(lambda x: x.split(' ')[0])

print(df['bodyType_1'].unique())
print(len(df['bodyType_1'].unique()))

In [ ]:
plt.figure(figsize=(10, 5))
order = df[df.train == 1].groupby('bodyType_1').price_log.median().sort_values().index

g = sns.boxplot(x = 'bodyType_1', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / bodyType_1_train')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

**2 Вариант**

In [ ]:
bodyType_test = list(df[df.train == 0].bodyType.unique())
bodyType_test

In [ ]:
df[df.train == 1].bodyType.apply(lambda x: x if x in bodyType_test else 'другой').unique()

Лимузины не попали в конечный список значений хотя присутствуют в тесте. Скорее всего значения принадлежащие лимузинам содержат лишнюю информацию, проверим

In [ ]:
df[df.train == 1][df[df.train == 1].bodyType.str.contains('лимузин')]

Наше предположение оправдалось, проверим данные по лимузинам из теста

In [ ]:
df[df.train == 0][df[df.train == 0].bodyType.str.contains('лимузин')]

Вышли те же самые лимузины pullman. Необходимо включить данные значения в наш датасет, не относя к значению "другой"

In [ ]:
df['bodyType_2'] = df.bodyType.apply(lambda x: x if x in bodyType_test or x == 'лимузин pullman' else 'другой')
df['bodyType_2'] = df.bodyType_2.apply(lambda x: x.split()[0] if x == 'лимузин pullman' else x)

In [ ]:
df['bodyType_2'].value_counts()

In [ ]:
print(df['bodyType_2'].unique())
print(len(df['bodyType_2'].unique()))

In [ ]:
df.sample(2)

In [ ]:
plt.figure(figsize=(15, 8))
order = df[df.train == 1].groupby('bodyType_2').price_log.median().sort_values().index

g = sns.boxplot(x = 'bodyType_2', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / bodyType_2_train')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

Интересно то, что визуально графики для bodyType_1 и bodyType_2 одинаковы за исключением значения *other* и значение тарга во втором случае по медиане находится не так высоко. Возможно удаление  *other* в дальнейшем сможет улучшить работу модели. Оставим это на потом

In [ ]:
# Внесем данные признаки в наш список

cat_cols.append('bodyType_1')
cat_cols.append('bodyType_2')

## 2. brand

In [ ]:
col_info(df['brand'])

In [ ]:
col_info(df[df.train == 1]['brand'])

In [ ]:
col_info(df[df.train == 0]['brand'])

In [ ]:
plt.figure(figsize=(17, 8))
order = df[df.train == 1].groupby('brand').price_log.median().sort_values().index

g = sns.boxplot(x = 'brand', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / brand_train')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

Cамыми дорогими авто в среднем оказались PORSCHE и MERCEDES, что весьма логично  учитывая статус брендов. Можно сделать вывод, что зависимость цены от бренда присутствует. Логичным решением будет сократить тренировочную часть до брендов из теста, а лишние вывести в отдельное значение *Other*, но перед этим можно создать новую фичу, на основании страны бренда и проверить зависимость. При неудовлетворительных или противоречивых результатах значения лишних стран так же поместим в *Other*

In [ ]:
df.brand.unique()

In [ ]:
# Создадим признак country_of_brand, показывающий принадлежность бренда к определенной стране производителю

countries = {
    'BMW': 'Germany',
    'AUDI': 'Germany',
    'CADILLAC' : 'USA',
    'CHERY' : 'China',
    'CHEVROLET' : 'USA',
    'CHRYSLER' : 'USA',
    'CITROEN' : 'France',
    'DAEWOO' : 'Korea',
    'DODGE' : 'USA',
    'FORD': 'USA',
    'GEELY' : 'China',
    'HONDA' : 'Japan',
    'HYUNDAI': 'Korea',
    'INFINITI' : 'Japan',
    'JAGUAR' : 'UK',
    'JEEP' : 'USA',
    'KIA' : 'Korea',
    'LEXUS' : 'Japan',
    'MAZDA' : 'Japan',
    'MERCEDES': 'Germany',
    'MINI' : 'UK',
    'MITSUBISHI': 'Japan',
    'NISSAN' : 'Japan',
    'OPEL' : 'Germany',
    'PEUGEOT': 'France',
    'PORSCHE' : 'Germany',
    'RENAULT' : 'France',
    'SKODA' : 'Czech Republic',
    'SUBARU' : 'Japan',
    'SUZUKI' : 'Japan',
    'TOYOTA': 'Japan',
    'VOLKSWAGEN': 'Germany',
    'VOLVO': 'Sweden',
    'LAND_ROVER': 'UK',
    'SSANG_YONG' : 'Korea',
    'GREAT_WALL' : 'China'
}

In [ ]:
df['country_of_brand'] = df['brand'].map(countries)

In [ ]:
plt.figure(figsize=(17, 8))
order = df[df.train == 1].groupby('country_of_brand').price_log.median().sort_values().index

g = sns.boxplot(x = 'country_of_brand', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / country_of_brand_tr')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

Как и предполагалось, самыми дорогими являются британские и немецкие авто, а самыми дешевыми китайские. Очевидно, что в данном случае зависимость цены от страны бренда есть.

In [ ]:
col_info(df[df.train == 1]['country_of_brand'])

In [ ]:
col_info(df[df.train == 0]['country_of_brand'])

Идея для фичи: Возможно имеет смысл разграничить бренды на премиум и массовый сегменты до отделения лишних значенй. Информацию о принадлежности к сегментам возьмем на сайте АВТОСТАТА www.autostat.ru/news/45243/

In [ ]:
premium = {
    'BMW': 1,
    'AUDI': 1,
    'CADILLAC' : 1,
    'CHERY' : 0,
    'CHEVROLET' : 0,
    'CHRYSLER' : 0,
    'CITROEN' : 0,
    'DAEWOO' : 0,
    'DODGE' : 0,
    'FORD': 0,
    'GEELY' : 0,
    'HONDA' : 0,
    'HYUNDAI': 0,
    'INFINITI' : 1,
    'JAGUAR' : 1,
    'JEEP' : 1,
    'KIA' : 0,
    'LEXUS' : 1,
    'MAZDA' : 0,
    'MERCEDES': 1,
    'MINI' : 1,
    'MITSUBISHI': 0,
    'NISSAN' : 0,
    'OPEL' : 0,
    'PEUGEOT': 0,
    'PORSCHE' : 1,
    'RENAULT' : 0,
    'SKODA' : 0,
    'SUBARU' : 0,
    'SUZUKI' : 0,
    'TOYOTA': 0,
    'VOLKSWAGEN': 0,
    'VOLVO': 1,
    'LAND_ROVER': 1,
    'SSANG_YONG' : 0,
    'GREAT_WALL' : 0
}

In [ ]:
df['premium'] = df['brand'].map(premium)
df.premium.value_counts()

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Premium cars in train and test')

_ = sns.countplot(x = 'premium', data = df[df.train == 1], ax=axes[0])
axes[0].set_title('TRAIN')

_ = sns.countplot(x = 'premium', data = df[df.train == 0], ax=axes[1])
axes[1].set_title('TEST')

plt.show()

In [ ]:
plt.figure(figsize=(8, 5))

g = sns.boxplot(x = 'premium', y = 'price_log', data = df[df.train == 1])
g.set_title('price / premium')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

Как видно на графике, зависимость цены от премиальности бренда присутствует. Автомобили премиум сегмента в среднем дороже автомобилей массового сегмента

Теперь определим лишние значения в признаках *brand* и *country_of_brand* тренировочной части в значение *other*

In [ ]:
# Признак brand

brand_test = list(df[df.train == 0].brand.unique())
brand_test

In [ ]:
df['brand'] = df.brand.apply(lambda x: x if x in brand_test else 'other')

In [ ]:
df['brand'].value_counts()

In [ ]:
plt.figure(figsize=(17, 8))
order = df[df.train == 1].groupby('brand').price_log.median().sort_values().index

g = sns.boxplot(x = 'brand', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / brand_train')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

In [ ]:
# Признак country_of_brand

country_test = list(df[df.train == 0].country_of_brand.unique())
country_test

In [ ]:
df['country_of_brand'] = df.country_of_brand.apply(lambda x: x if x in country_test else 'other')

In [ ]:
df['country_of_brand'].value_counts()

In [ ]:
plt.figure(figsize=(17, 8))
order = df[df.train == 1].groupby('country_of_brand').price_log.median().sort_values().index

g = sns.boxplot(x = 'country_of_brand', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / country_of_brand_tr')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Не уверен, что это была хорошая идея, медианные цены на графике не сильно разнятся в зависимости от страны после изменения признака (хотя до него на графике очевидно зависимость присутствовала). Пока оставим данный признак. Впоследствии проверим работу модели с ним и без.

In [ ]:
# Внесем данные признаки в наши списки

cat_cols.append('brand')
cat_cols.append('country_of_brand')
bin_cols.append('premium')

## 3. color

In [ ]:
col_info(df.color)

In [ ]:
col_info(df[df.train == 0].color)

In [ ]:
plt.figure(figsize=(15, 8))
order = df[df.train == 1].groupby('color').price_log.median().sort_values().index

g = sns.boxplot(x = 'color', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / color')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels(), rotation = 90)

plt.show()

In [ ]:
# Внесем данный признак в наши списки

cat_cols.append('color')

## 4. fuelType

In [ ]:
col_info(df.fuelType)

In [ ]:
col_info(df[df.train==0].fuelType)

In [ ]:
plt.figure(figsize=(10, 5))
order = df[df.train == 1].groupby('fuelType').price_log.median().sort_values().index

g = sns.boxplot(x = 'fuelType', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / fuelType')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

In [ ]:
# Внесем данный признак в наши списки

cat_cols.append('fuelType')

## 5. modelDate

In [ ]:
# train

col_info(df[df.train == 1].modelDate)

In [ ]:
# test

col_info(df[df.train == 0].modelDate)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.modelDate, ax=ax[0], bins=30)
_ = sns.histplot(df[df.train == 1].modelDate, ax=ax[1], bins=30)
_ = sns.histplot(df[df.train == 0].modelDate, ax=ax[2], bins=30)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

Распределение данных в трейне и тесте примерно одинаковое

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'modelDate', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=10)
g.set_title('price / modelDate')
g.set_ylabel('price_log')
g.set_xticks(np.linspace(1900, 2020, 21))
plt.xticks(rotation=45)
plt.show()

Как видим, начиная приблизительно с 1980 года зависимость линейная (чем новее модель авто, тем оно дороже). Значения ниже 1980 с высоким ценником скорее всего автораритеты. Проверим

In [ ]:
df[(df.train == 1) & (df.modelDate < 1980) & (df.price_log > 12)]

И правда, по описанию некоторых экземпляров можно понять, что эти авто - раритеты

In [ ]:
df[(df.train == 0) & (df.modelDate < 1980)]

К сожалению, пока что я не понимаю по каким критериям отделить раритеты (Возможно я мог бы в трейне создать признак на основании года модели (меньше 1980), цены, а так же слов встречающихся в признаке description а после этого обучить модель и предсказать значение данного признака в test)

In [ ]:
# Внесем данный признак в наши списки

num_cols.append('modelDate')

## 6. productionDate

In [ ]:
# train

col_info(df[df.train == 1].productionDate)

In [ ]:
# test

col_info(df[df.train == 0].productionDate)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.productionDate, ax=ax[0], bins=30)
_ = sns.histplot(df[df.train == 1].productionDate, ax=ax[1], bins=30)
_ = sns.histplot(df[df.train == 0].productionDate, ax=ax[2], bins=30)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'productionDate', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=10)
g.set_title('price / productionDate')
g.set_ylabel('price_log')
g.set_xticks(np.linspace(1900, 2020, 21))
plt.xticks(rotation=45)
plt.show()

В данном случае линейная зависимость приблизительно с 1985 еще сильнее по сравнению с modelDate, за исключением некоторых старых, предположительно отреставрированных раритетных авто. Чем позже год производства, тем дороже авто.

Создадим признак отвечающий за вовраст автомобиля (2020 год так же учитываем за целый)

In [ ]:
df['car_age'] = (NOWDAYS - df.productionDate) + 1

In [ ]:
_ = sns.histplot(df.car_age, bins=30)

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'car_age', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=10)
g.set_title('price / car_age')
g.set_ylabel('price_log')
plt.show()

Чем старше авто, тем, как правило, ниже цена

Создадим также признак в виде разницы между датой выпуска модели и датой выпуска непосредственно авто

In [ ]:
df['model_prod_date_delta'] = df.productionDate - df.modelDate

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'model_prod_date_delta', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=15)
g.set_title('price / model_prod_date_delta')
g.set_ylabel('price_log')
plt.show()

По графику видно, что чем больше разброс между датой выпуска и производства, тем меньше в среднем стоимость авто

In [ ]:
# Внесем данные признаки в наши списки

num_cols.append('productionDate')
num_cols.append('car_age')
num_cols.append('model_prod_date_delta')

## 7. numberOfDoors

In [ ]:
# train

col_info(df[df.train == 1].numberOfDoors)

In [ ]:
# test

col_info(df[df.train == 0].numberOfDoors)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.countplot(df.numberOfDoors, ax=ax[0])
_ = sns.countplot(df[df.train == 1].numberOfDoors, ax=ax[1])
_ = sns.countplot(df[df.train == 0].numberOfDoors, ax=ax[2])

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
order = df[df.train == 1].groupby('numberOfDoors').price_log.median().sort_values().index

g = sns.boxplot(x = 'numberOfDoors', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / numberOfDoors')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Как видно из графика, трех-четырехдверные авто чаще всего самые дешевые. Самыми же дорогими являются двухдверные авто, куда предположительно входят спорткары, и одно авто без двери. Проверим что это такое

In [ ]:
df[(df.train == 1) & (df.numberOfDoors == 0)]

Раритеное и довольно дорогое авто, оставим его, так как в тесте так же есть значение с таким же количеством дверей

In [ ]:
# Внесем данный признак в наши списки

num_cols.append('numberOfDoors')

## 8. name

In [ ]:
# train

col_info(df[df.train == 1].name)

In [ ]:
# test

col_info(df[df.train == 0].name)

Данный признак является лишь компиляцией других и включает в себя тип коробки передач, количество л.с., объем двигателя, тип привода. Удалим данный признак из нашего датасета.

In [ ]:
df.drop(columns='name', inplace=True)

## 9. vehicleConfiguration

In [ ]:
# train

col_info(df[df.train == 1].vehicleConfiguration)

То же, что и признак name - удаляем

In [ ]:
df.drop(columns='vehicleConfiguration', inplace=True)

## 10. vehicleTransmission

In [ ]:
# train

col_info(df[df.train == 1].vehicleTransmission)

In [ ]:
# test

col_info(df[df.train == 0].vehicleTransmission)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.countplot(df.vehicleTransmission, ax=ax[0])
_ = sns.countplot(df[df.train == 1].vehicleTransmission, ax=ax[1])
_ = sns.countplot(df[df.train == 0].vehicleTransmission, ax=ax[2])

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=30)

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
order = df[df.train == 1].groupby('vehicleTransmission').price_log.median().sort_values().index

g = sns.boxplot(x = 'vehicleTransmission', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / vehicleTransmission')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

In [ ]:
# Внесем данный признак в наши списки

cat_cols.append('vehicleTransmission')

## 11. enginePower

In [ ]:
# train

col_info(df[df.train == 1].enginePower)

In [ ]:
# test

col_info(df[df.train == 0].enginePower)

In [ ]:
# Приведем данный признак к числовому значению

df['enginePower'] = df.enginePower.apply(lambda x: int(x.split(' ')[0]))

In [ ]:
col_info(df[df.train == 1].enginePower)

In [ ]:
col_info(df[df.train == 0].enginePower)

Оставим значения мощности в пределах минимума и максимума значений тестовой части

In [ ]:
test_power_max = df[df.train == 0].enginePower.max()
test_power_min = df[df.train == 0].enginePower.min()

print(test_power_max)
print(test_power_min)

In [ ]:
df = df[(df.enginePower <= test_power_max) & (df.enginePower >= test_power_min)]

In [ ]:
col_info(df.enginePower)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.enginePower, ax=ax[0], bins=20)
_ = sns.histplot(df[df.train == 1].enginePower, ax=ax[1], bins=20)
_ = sns.histplot(df[df.train == 0].enginePower, ax=ax[2], bins=20)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'enginePower', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=5)
g.set_title('price / enginePower')
g.set_ylabel('price_log')
plt.show()

In [ ]:
df.enginePower.value_counts(bins=3)

На гистограмме видим скорее всего логнормальное распределение. Создадим новый признак enginePower_log

In [ ]:
df['enginePower_log'] = np.log(df.enginePower)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.enginePower_log, ax=ax[0], bins=20)
_ = sns.histplot(df[df.train == 1].enginePower_log, ax=ax[1], bins=20)
_ = sns.histplot(df[df.train == 0].enginePower_log, ax=ax[2], bins=20)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'enginePower_log', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=5)
g.set_title('price / enginePower_log')
g.set_ylabel('price_log')
plt.show()

Итого по признаку:
1. Большинство значений варьируется в пределах от 50 до 250.
2. Признак имеет линейную зависимость (чем больше значение, тем больше цена авто)

In [ ]:
# Внесем данные признаки в наши списки
num_cols.append('enginePower')
num_cols.append('enginePower_log')

## 12. engineDisplacement

Перед работой над признаком переведем значения из л в см3

In [ ]:
# Переведем тип признака в float

df['engineDisplacement'] = df.engineDisplacement.astype('float') * 1000
df['engineDisplacement'] 

In [ ]:
# train

col_info(df[df.train == 1].engineDisplacement)

In [ ]:
# test

col_info(df[df.train == 0].engineDisplacement)

Количество в пропусках train и test подозрительно знакомы, скорее всего это электрокары, где литраж двигателя указан не был

In [ ]:
df[(df.train == 1) & (df.fuelType == 'электро')].engineDisplacement

In [ ]:
df[(df.train == 0) & (df.fuelType == 'электро')].engineDisplacement

In [ ]:
df[(df.fuelType == 'электро')]

Выявим наиболее часто встречающиеся параметры и интервалы в значениях и по ним найдем авто схожие с электрокарами по параметрам. На основе их объема двигателя заполним столбец. (Фильтровать будем по тесту)

**признак bodyType**

In [ ]:
print(df[(df.fuelType == 'электро')].bodyType_1.value_counts())
print()
print(df[(df.fuelType == 'электро')].bodyType_2.value_counts())

In [ ]:
print(df[(df.train==0) & (df.fuelType == 'электро')].bodyType_1.value_counts())
print()
print(df[(df.train==0) & (df.fuelType == 'электро')].bodyType_2.value_counts())

Наиболее встречающиеся значения тут это хэтчбек и внедорожник, фильтровать будем про признаку bodyType_1 ибо числа не различаются. Попробуем для начала отфильтровать по значению хэтчбек

**признак brand**

In [ ]:
df[(df.fuelType == 'электро')].brand.value_counts()

In [ ]:
df[(df.train==0) & (df.fuelType == 'электро')].brand.value_counts()

In [ ]:
pd.pivot_table(data=df[(df.train==1) & (df.fuelType == 'электро')], values='price', columns='brand', aggfunc='mean')

Для начала отфильтруем по трем брендам (NISSAN, BMW и AUDI)

**признак numberOfDoors**

In [ ]:
df[(df.fuelType == 'электро')].numberOfDoors.value_counts()

Возьмем значение 5

**признак enginePower**

In [ ]:
df[(df.fuelType == 'электро')].enginePower.value_counts()

In [ ]:
df[(df.train==0) & (df.fuelType == 'электро')].enginePower.value_counts()

Значения 109, 170 и 408

**признак modelDate**

In [ ]:
df[(df.fuelType == 'электро')].modelDate.value_counts(bins=2)

In [ ]:
df[(df.train==0) & (df.fuelType == 'электро')].modelDate.value_counts(bins=2)

Возьмем значения с 2009 по 2014 год

In [ ]:
# Фильтруем

df[(df.bodyType_1 == 'хэтчбек') & ((df.brand == 'AUDI') | (df.brand == 'BMW') | (df.brand == 'NISSAN')) \
   & (df.modelDate >= 2009) & (df.modelDate <= 2014) & (df.numberOfDoors == 5) & ((df.enginePower == 109) \
   | (df.enginePower == 170) | (df.enginePower == 408)) & (df.fuelType != 'электро')].engineDisplacement.value_counts()

In [ ]:
# На всякий случай проверим тестовую часть

df[(df.train == 0) & (df.bodyType_1 == 'хэтчбек') & ((df.brand == 'AUDI') | (df.brand == 'BMW') | (df.brand == 'NISSAN')) \
   & (df.modelDate >= 2009) & (df.modelDate <= 2014) & (df.numberOfDoors == 5) & ((df.enginePower == 109) \
   | (df.enginePower == 170) | (df.enginePower == 408)) & (df.fuelType != 'электро')].engineDisplacement.value_counts()

In [ ]:
# Подставим полученное значение заместо пропусков и нулей

df.engineDisplacement.fillna(1600, inplace=True)
df['engineDisplacement'] = df.engineDisplacement.apply(lambda x: 1600 if x == 0 else x)

In [ ]:
# train

col_info(df[df.train == 1].engineDisplacement)

In [ ]:
# test

col_info(df[df.train == 0].engineDisplacement)

Посмотрим что за авто, мощность которых выше максимальной в тесте

In [ ]:
test_displ_max = df[df.train == 0].engineDisplacement.max()
test_displ_max

In [ ]:
df[df.engineDisplacement > 6600]

Удалим данные авто из нашего датасета

In [ ]:
df = df[df.engineDisplacement <= test_displ_max]

In [ ]:
# Проверим

df.engineDisplacement.describe()

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.engineDisplacement, ax=ax[0], bins=15)
_ = sns.histplot(df[df.train == 1].engineDisplacement, ax=ax[1], bins=15)
_ = sns.histplot(df[df.train == 0].engineDisplacement, ax=ax[2], bins=15)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'engineDisplacement', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=5)
g.set_title('price / engineDisplacement')
g.set_ylabel('price_log')
plt.show()

На графике видна небольшая зависимость цены от объема двигателя. Чем больше объем, тем дороже авто в среднем 

In [ ]:
# Внесем данные признаки в наши списки
num_cols.append('engineDisplacement')

## 13. mileage

In [ ]:
col_info(df.mileage)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.mileage, ax=ax[0], bins=15)
_ = sns.histplot(df[df.train == 1].mileage, ax=ax[1], bins=15)
_ = sns.histplot(df[df.train == 0].mileage, ax=ax[2], bins=15)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
len(df[df.mileage == 0])

В датасете слишком много нулевых значений пробега. Но пробег сам по себе даже у новой машины не может быть нулевым и находится в интервале 15-20 км. Заменим все значения меньше 15 на случайные в данном интервале, но перед этим создадим новый признак, который будет обозначением нового авто без пробега

In [ ]:
df['new_car?'] = df.mileage.apply(lambda x: 1 if x == 0 else 0)

In [ ]:
sns.countplot(df['new_car?'])

plt.show()

In [ ]:
# Присвоим значениям меньше 15 км пробег от 15 до 20 км случайным образом

np.random.seed(RANDOM_SEED)

df['mileage'] = df.mileage.apply(lambda x: np.random.randint(15, 20) if x < 15 else x)

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.mileage, ax=ax[0], bins=20)
_ = sns.histplot(df[df.train == 1].mileage, ax=ax[1], bins=20)
_ = sns.histplot(df[df.train == 0].mileage, ax=ax[2], bins=20)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'mileage', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=1)
g.set_title('price / mileage')
g.set_ylabel('price_log')
plt.show()

В данном случае чем больше пробег, тем, как правило, ниже цена.

Создадим признак mileage_per_age, который будет показывать, как интенсивно использовали автомобиль 

In [ ]:
df['mileage_per_age'] = df.mileage / df.car_age

col_info(df['mileage_per_age'])

Удалим выбросы из трейна на оснований границ теста

In [ ]:
# Получим максимальные и минимальные значения теста

test_mileage_per_age_max = df[df.train == 0].mileage_per_age.max()
test_mileage_per_age_min = df[df.train == 0].mileage_per_age.min()

print(test_mileage_per_age_max)
print(test_mileage_per_age_min)

In [ ]:
# Оставим значения по границам максимума и минимума

df = df[(df.mileage_per_age <= test_mileage_per_age_max) & (df.mileage_per_age >= test_mileage_per_age_min)]

col_info(df.mileage_per_age)

Посмотрим как изменились графики  по mileage и mileage_per_age

**mileage**

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.mileage, ax=ax[0], bins=15)
_ = sns.histplot(df[df.train == 1].mileage, ax=ax[1], bins=15)
_ = sns.histplot(df[df.train == 0].mileage, ax=ax[2], bins=15)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'mileage', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=1)
g.set_title('price / mileage')
g.set_ylabel('price_log')
plt.show()

**mileage_per_age**

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 5)

_ = sns.histplot(df.mileage_per_age, ax=ax[0], bins=15)
_ = sns.histplot(df[df.train == 1].mileage_per_age, ax=ax[1], bins=15)
_ = sns.histplot(df[df.train == 0].mileage_per_age, ax=ax[2], bins=15)

ax[0].set_title('Both')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

g = sns.scatterplot(x = 'mileage_per_age', y = 'price_log', data = df[df.train == 1], alpha=0.2, s=1)
g.set_title('price / mileage_per_age')
g.set_ylabel('price_log')
plt.show()

Как видно из графика, интенсивность не сильно влияет на стоимость авто. Но мы посмотрим на это еще раз на этапе оценки полезности признаков

In [ ]:
# Внесем данные признаки в наши списки
num_cols.append('mileage')
num_cols.append('mileage_per_age')
bin_cols.append('new_car?')

## 14. Привод

In [ ]:
col_info(df['Привод'])

In [ ]:
# test

col_info(df[df.train==0]['Привод'])

In [ ]:
plt.figure(figsize=(17, 8))
order = df[df.train == 1].groupby('Привод').price_log.median().sort_values().index

g = sns.boxplot(x = 'Привод', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / Привод')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Как видно из графика, авто с полным приводом стоят дороже всего, в то время как авто с передним приводом дешевле всего

In [ ]:
# Внесем данные признаки в наши списки

cat_cols.append('Привод')

## 15. Руль

In [ ]:
col_info(df['Руль'])

In [ ]:
# test

col_info(df[df.train==0]['Руль'])

In [ ]:
plt.figure(figsize=(10, 6))
order = df[df.train == 1].groupby('Руль').price_log.median().sort_values().index

g = sns.boxplot(x = 'Руль', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / Руль')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Леворульные авто как правило дороже праворульных

In [ ]:
# Зададим значение для левого руля - 1, а для правого - 0

df['Руль'] = df['Руль'].apply(lambda x: 1 if x == 'Левый' else 0)

In [ ]:
# Внесем данные признаки в наши списки

bin_cols.append('Руль')

## 16. Владельцы

In [ ]:
col_info(df['Владельцы'])

In [ ]:
# test

col_info(df[df.train==0]['Владельцы'])

Заменим строковые значения на числовые

In [ ]:
df['Владельцы'].unique()

In [ ]:
def num_from_owners(row):
    '''Функция вытаскивает число из строки'''
    
    row = str(row)
    result = re.findall('\d', row)
    if result == []:
        return None
    return result[0]

In [ ]:
df['Владельцы'] = df['Владельцы'].apply(num_from_owners)

In [ ]:
# Посмотрим на пропуски

df.loc[df['Владельцы'].isna()]

In [ ]:
# По году производства можно судить что это новые авто, проверим

print(df.loc[df['Владельцы'].isna()]['new_car?'].unique())
print()
print(df.loc[df['Владельцы'].isna()].productionDate.describe())

Предположение подтвердилось, все авто с пропущенными значениями новые. Заменим тип значений на int и, добавив новое значение 0, поставим его вместо пропусков

In [ ]:
df['Владельцы'].fillna(0, inplace=True)

In [ ]:
df['Владельцы'] = df['Владельцы'].astype('int')

In [ ]:
plt.figure(figsize=(15, 8))
order = df[df.train == 1].groupby('Владельцы').price_log.median().sort_values().index

g = sns.boxplot(x = 'Владельцы', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / Владельцы')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

График выглядит вполне логично. Чем меньше владельцев у авто, тем оно дороже. Фактически данный признак является категориальным порядковым.

In [ ]:
# Внесем данные признаки в наши списки

ordinal_cat_cols.append('Владельцы')

## 17. ПТС

In [ ]:
col_info(df['ПТС'])

In [ ]:
# test

col_info(df[df.train==0]['ПТС'])

In [ ]:
# Посмотрим на пропуски

df.loc[df['ПТС'].isna()]

In [ ]:
# По году производства можно судить что это новые авто, проверим

print(df.loc[df['ПТС'].isna()]['new_car?'].value_counts())
print()
print(df.loc[df['ПТС'].isna()].productionDate.describe())

324 новых авто и 1 выброс, посмотрим на него

In [ ]:
df.loc[df['ПТС'].isna()][df['new_car?'] == 0]

In [ ]:
df.loc[df['ПТС'].isna()][df['new_car?'] == 0].index

In [ ]:
# Так как данное значение находится в тесте, заменим его на самое частовстречаемое (Оригинал), а остальные заменим новым значение 'нет'

df.loc[95802, 'ПТС'] = 'Оригинал'
df['ПТС'].fillna('Нет', inplace=True)

In [ ]:
# Проверим

col_info(df['ПТС'])

In [ ]:
plt.figure(figsize=(15, 8))
order = df[df.train == 1].groupby('ПТС').price_log.median().sort_values().index

g = sns.boxplot(x = 'ПТС', y = 'price_log', data = df[df.train == 1], order=order)
g.set_title('price / ПТС')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с оригиналом птс дороже чем с дупликатом, в то же время авто без птс (новое) стоит дороже всего.

In [ ]:
# Внесем данные признаки в наши списки

cat_cols.append('ПТС')

## 18. Владение

In [ ]:
col_info(df['Владение'])

In [ ]:
col_info(df[df.train==0]['Владение'])

In [ ]:
len(df['Владение'])

В признаке слишком много пропусков. Не знаю как можно его использовать в данном случае. Удалим.

In [ ]:
# Дропаем признак владения

df.drop(columns='Владение', inplace=True)

## 19. model_name

In [ ]:
col_info(df.model_name)

In [ ]:
# train

col_info(df[df.train==1].model_name)

In [ ]:
# test

col_info(df[df.train==0].model_name)

Уникальных значений модели авто в тесте в два раза меньше чем в трейне. Оставим в тренировочной части значения по тесту, остальные закинем в значение *other*

In [ ]:
model_name_test = list(df[df.train==0].model_name.unique())

In [ ]:
df['model_name'] = df.model_name.apply(lambda x: x if x in model_name_test else 'other')

In [ ]:
# Внесем данные признаки в наши списки

cat_cols.append('model_name')

## 20. description

In [ ]:
# train

col_info(df[df.train==1].description)

In [ ]:
# test

col_info(df[df.train==0].description)

In [ ]:
# заполним пропуски

df['description'] = df.description.fillna(' ')

In [ ]:
# Для удобства присвоим столбец новой переменной

data = df['description']

In [ ]:
data

In [ ]:
# Переведем текст в нижний регистр

data = data.str.lower()

In [ ]:
# Удалим символ переноса строки, табуляции и странную точку найденную в данных

symbols = ['\n', '\t', '•']

for pat in symbols:
    data = data.str.replace(pat,' ')

In [ ]:
# Удалим все численные обозначения

for pat in range(10):
    data = data.str.replace(str(pat),' ')

In [ ]:
# Удалим иностранные символы (латынь)

data = data.str.replace(r'[a-z]',' ')

In [ ]:
# Удалим знаки пунктуации

for pat in punctuation:
    data = data.str.replace(pat,' ')

In [ ]:
# Опытным путем выяснилось, что необходимо повторно пройтись данной функцией для очистки данных от всех "не буквенных" символов

def clean_symbs(data):
    clean = re.sub(r"[^\w]", " ", data)
    return clean

data = data.apply(clean_symbs)

In [ ]:
# Выбираем только те строки, которые не состоят полностью из пробельных символов (не пустые)

data = data[~data.str.isspace()]

In [ ]:
def without_space(data):
    '''Функция для очистки строки от лишних пробелов'''
    a = re.sub(r'\s+', ' ', data)
    return a

data = data.apply(without_space)

In [ ]:
# Разделим строки на токены

data = data.str.strip(' ')

In [ ]:
# Создаем лист стоп слов

russian_stopwords = stopwords.words("russian")

In [ ]:
mystem = Mystem() 

# В функции с предобработкой пришлось снова писать код для очистки от пунктуации, так как некоторые символы до конца не хотели удаляться

def preprocess_text(text):
    '''Функция для лемматизации текста и очистки от стоп-слов'''
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " "\
              and token.strip() not in punctuation]
  
    return tokens

In [ ]:
counter = Counter()
def count_words(sentence):
    global counter
    for x in sentence:
        counter[x] += 1

In [ ]:
data.apply(count_words)

In [ ]:
# Посмотрим на наиболее встречающиеся значения, а так же проверим результат работы функции

counter.most_common(n=500)

In [ ]:
df['description_tokens'] = data

In [ ]:
df.description_tokens.isna().sum()

In [ ]:
df['description_tokens'].fillna('', inplace=True)
df['description_tokens']

In [ ]:
# Посмотрим на самые частовстречающиеся теги

wordcloud = WordCloud(
    background_color='white',
    max_words=300,
    max_font_size=200, 
    width=1000, height=800,
    random_state=RANDOM_SEED,
).generate(" ".join(df['description_tokens'].astype(str)))

plt.figure(figsize = (12, 14), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [ ]:
# Создадим новые признаки на основании сочетаний слов, которые могут влиять на цену авто

# Подушка безопасности
df['airbag'] = df.description_tokens.apply(lambda x: 1 if ('подушка' and 'безопасность') in x else 0)

# Отличное состояние
df['excellent_condition'] = df.description_tokens.apply(lambda x: 1 if ('отличный' and 'состояние') in x else 0)

# Легкосплавные диски
df['alloy_wheels'] = df.description_tokens.apply(lambda x: 1 if ('легкосплавный' and 'диск') in x else 0)

# Обогрев зеркал
df['heated_mirrors'] = df.description_tokens.apply(lambda x: 1 if ('обогрев' and 'зеркало') in x else 0)

# Центральный замок
df['central_locking'] = df.description_tokens.apply(lambda x: 1 if ('центральный' and 'замок') in x else 0)

# Бортовой компьютер
df['on-board_computer'] = df.description_tokens.apply(lambda x: 1 if ('бортовой' and 'компьютер') in x else 0)

# АБС
df['abs'] = df.description_tokens.apply(lambda x: 1 if ('антиблокировочный' and 'система') in x else 0)

# Датчик света
df['light_sensor'] = df.description_tokens.apply(lambda x: 1 if ('датчик' and 'свет') in x else 0)

# Обивка салона
df['upholstery'] = df.description_tokens.apply(lambda x: 1 if ('обивка' and 'салон') in x else 0)

# Подогрев сидения
df['heated_seat'] = df.description_tokens.apply(lambda x: 1 if ('подогрев' and 'сидение') in x else 0)

# Датчик дождя
df['rain_sensor'] = df.description_tokens.apply(lambda x: 1 if ('датчик' and 'дождь') in x else 0)

# Официальный диллер
df['official_dealer'] = df.description_tokens.apply(lambda x: 1 if ('официальный' and 'диллер') in x else 0)

# Хорошее состояние
df['good_condition'] = df.description_tokens.apply(lambda x: 1 if ('хороший' and 'состояние') in x else 0)

# Усилитель руля
df['power_steering'] = df.description_tokens.apply(lambda x: 1 if ('усилитель' and 'руль') in x else 0)

# Круиз контроль
df['сruise_control'] = df.description_tokens.apply(lambda x: 1 if ('круиз' and 'контроль') in x else 0)

# Климат контроль
df['climate_control'] = df.description_tokens.apply(lambda x: 1 if ('климат' and 'контроль') in x else 0)

# Светодиодные фары
df['led_lights'] = df.description_tokens.apply(lambda x: 1 if ('светодиодный' and 'фара') in x else 0)

# Противотуманные фары
df['fog_lights'] = df.description_tokens.apply(lambda x: 1 if ('противотуманный' and 'фара') in x else 0)

# Камера заднего вида
df['rv_camera'] = df.description_tokens.apply(lambda x: 1 if ('камера' and 'задний') in x else 0)

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'airbag', y = 'price_log', data = df[df.train == 1])
g.set_title('price / airbag')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием в описании наличия подушки безопасности обычно стоят дороже чем без него

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'excellent_condition', y = 'price_log', data = df[df.train == 1])
g.set_title('price / excellent_condition')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Усредненные значения цен практически не отличаются. В случае отличного состояния цена чуть ниже и меньше разброс стоимости

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'alloy_wheels', y = 'price_log', data = df[df.train == 1])
g.set_title('price / alloy_wheels')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Чаще всего авто с легкосплавными дисками стоит дороже

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'heated_mirrors', y = 'price_log', data = df[df.train == 1])
g.set_title('price / heated_mirrors')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Чаще всего авто с обогревом зеркал стоит дороже

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'central_locking', y = 'price_log', data = df[df.train == 1])
g.set_title('price / central_locking')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием центрального замка дороже

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'on-board_computer', y = 'price_log', data = df[df.train == 1])
g.set_title('price / on-board_computer')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием бортового компьютера зачастую дороже чем без него

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'abs', y = 'price_log', data = df[df.train == 1])
g.set_title('price / abs')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием антиблокировочной системы зачастую дороже чем без него

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'light_sensor', y = 'price_log', data = df[df.train == 1])
g.set_title('price / light_sensor')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с датчиком света также выше по стоимости чем без него

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'upholstery', y = 'price_log', data = df[df.train == 1])
g.set_title('price / upholstery')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием обивки салона дороже

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'heated_seat', y = 'price_log', data = df[df.train == 1])
g.set_title('price / heated_seat')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием подогрева сидений дороже

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'rain_sensor', y = 'price_log', data = df[df.train == 1])
g.set_title('price / rain_sensor')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием датчика дождя выше по стоимости

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'official_dealer', y = 'price_log', data = df[df.train == 1])
g.set_title('price / official_dealer')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминанием официального диллера в описании стоит дороже

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'good_condition', y = 'price_log', data = df[df.train == 1])
g.set_title('price / good_condition')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Та же ситуация, что и с "отличным состоянием". Скорее всего придется удалить оба признака

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'power_steering', y = 'price_log', data = df[df.train == 1])
g.set_title('price / power_steering')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с усилителем руля выше по стоимости

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'сruise_control', y = 'price_log', data = df[df.train == 1])
g.set_title('price / сruise_control')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с круиз-контролем выше в цене

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'climate_control', y = 'price_log', data = df[df.train == 1])
g.set_title('price / climate_control')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Авто с упоминание климат-контроля выше по стоимости

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'led_lights', y = 'price_log', data = df[df.train == 1])
g.set_title('price / led_lights')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Присутствие светодиодных фар в описании чаще всего повышает стоимость авто

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'fog_lights', y = 'price_log', data = df[df.train == 1])
g.set_title('price / fog_lights')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Такая же ситуация как и со светодиодными фарами

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.boxplot(x = 'rv_camera', y = 'price_log', data = df[df.train == 1])
g.set_title('price / rv_camera')
g.set_ylabel('price_log')
g.set_xticklabels(g.get_xticklabels())

plt.show()

Упоминание камеры заднего вида так же повышает цену автомобиля

In [ ]:
# Добавим так же новые признаки для количества слов, символов и средней длины слова
# Количество слов в строке
df['word_count'] = df.description_tokens.apply(lambda x: len(x))

# Количество символов в строке
df['char_count'] = df.description_tokens.apply(lambda x: sum(len(word) for word in x))

# Средняя длина слова в строке
df['avg_word_length'] = df['char_count'] / df['word_count']
df.avg_word_length.fillna(0, inplace=True)

In [ ]:
# Удалим признаки отличного и хорошего состояния, т.к. они практически не влияют на цену

df.drop(columns=['excellent_condition', 'good_condition'], inplace=True)

In [ ]:
df.description_tokens.apply(count_words)

In [ ]:
counter.most_common(n=100)

In [ ]:
def find_threshold(value=3):
    '''Функция устанавливает порог по количеству уникального слова в признаке и выводит индекс порогового значения функции counter'''
    for n, (word, count) in enumerate(counter.most_common()):
        if count == value:
            return n

In [ ]:
# Найдем индекс по порогу в 150 слов, чтобы исключить мусор и грамматические ошибки

find_threshold(150)

In [ ]:
# Создадим словарь отфильтрованных слов

words_filtred = dict(counter.most_common(4586))

In [ ]:
# Оставим в нашем признаке только те слова, которые прошли по порогу

df.description_tokens = df.description_tokens.apply(lambda x: [word for word in x if word in words_filtred])

In [ ]:
# При помощи TfidfVectorizer создадим матрицу tf-idf значений для каждого слова

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform([" ".join(x) for x in df.description_tokens.values])

In [ ]:
# Для наглядности результата создадим датафрейм

tfidf_tokens = tfidf_vectorizer.get_feature_names()
tfidf_array = tfidf.toarray()
df_tfidfvect = pd.DataFrame(data = tfidf_array,columns = tfidf_tokens)

In [ ]:
df_tfidfvect

In [ ]:
# Создадим модель TSVD с 10 компонентами

TSVD = TruncatedSVD(n_components=10, random_state=RANDOM_SEED)

TSVD.fit(tfidf_array)

In [ ]:
# Воспользуемся методом локтя, чтобы вычислить оптимальное число компонент

plt.plot(TSVD.explained_variance_ratio_)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.xlim((0, 9))
plt.show()

In [ ]:
# Снова создадим модель TSVD уже с найденным количеством компонент и обучим нашу матрицу

TSVD = TruncatedSVD(n_components=4, random_state=RANDOM_SEED)

TSVD.fit(tfidf_array)

In [ ]:
TSVD.explained_variance_ratio_

In [ ]:
# Уменьшаем размерность нашей tf-idf матрицы до 4 столбцов

TSVD_features = TSVD.transform(tfidf_array)

In [ ]:
TSVD_features

In [ ]:
# Добавим новые признаки в списки

bin_cols.extend(['airbag', 'alloy_wheels', 'heated_mirrors', 'central_locking', 'on-board_computer', 'abs', 
                 'light_sensor', 'upholstery', 'heated_seat', 'rain_sensor', 'official_dealer', 'power_steering', 
                 'сruise_control', 'climate_control', 'led_lights', 'fog_lights', 'rv_camera'])
num_cols.extend(['word_count', 'char_count', 'avg_word_length'])

##21. complectation_dict

In [ ]:
col_info(df.complectation_dict)

In [ ]:
# Удалим данный признак так как в тесте было изначально слишком много пропусков, плюс на данный момент я не представляю как сейчас можно с пользой его обработать

df.drop(columns=['complectation_dict'], inplace=True)

## Сохранение датасета после EDA

In [ ]:
df.sample(3)

In [ ]:
cols_to_drop = ['bodyType', 'engineDisplacement2', 'description', 'description_tokens']
                
for col in cols_to_drop:
    if col in list(df.columns):
        df.drop(columns=col, inplace=True)

In [ ]:
df.sample(3)

In [ ]:
target_cols = ['price', 'price_log']

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df[df.train==1][num_cols+target_cols].corr().abs(), vmin=-1, vmax=1, annot=True)
plt.show()

modelDate, productionDate и car_age сильно скореллированны между собой, так же как и enginePower, enginePower_log  и word_count, char_count. Посмотрим значимость данных признаков через ANOVA f-test , чтобы решить, какой из них удалить.

In [ ]:
imp_num = pd.Series(f_classif(df[df.train==1][num_cols], df[df.train==1]['price_log'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость непрерывных переменных по ANOVA F-test')

plt.show()

Удалим признаки productionDate и enginePower из нашего датасета

In [ ]:
num_cols.remove('productionDate')
num_cols.remove('enginePower')
num_cols.remove('word_count')
num_cols.remove('car_age')

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df[df.train==1][num_cols+target_cols].corr().abs(), vmin=-1, vmax=1, annot=True)
plt.show()

In [ ]:
train_col = ['train']

In [ ]:
sum_cols = num_cols+cat_cols+bin_cols+ordinal_cat_cols+target_cols+train_col

df_1 = df.loc[:, sum_cols].copy()

# Data Preprocessing

In [ ]:
df_1[num_cols].info()

In [ ]:
df_1[num_cols].values

In [ ]:
# Стандартизируем наши числовые признаки

scaler = StandardScaler()
standart_nums = scaler.fit_transform(df_1[num_cols])

In [ ]:
standart_nums

In [ ]:
# Закодируем нашу порядковый категориальный признак при помощи OrdinalEncoder

oenc = OrdinalEncoder()

oenc.fit(df_1[ordinal_cat_cols])

In [ ]:
ordinal_features = oenc.transform(df_1[ordinal_cat_cols])

In [ ]:
oenc.categories_

In [ ]:
# Закодируем наши категориальные переменные при помощи one-hot encoding

ohe_1 = OneHotEncoder()
ohe_2 = OneHotEncoder()

In [ ]:
cat_cols_1 = []
cat_cols_2 = []

for word in cat_cols:
    if word != 'bodyType_2':
        cat_cols_1.append(word)
    if word != 'bodyType_1':
        cat_cols_2.append(word)

In [ ]:
cat_features_1 = ohe_1.fit_transform(df_1[cat_cols_1])
cat_features_2 = ohe_2.fit_transform(df_1[cat_cols_2])

In [ ]:
# Индексы тренировочных данных

df_1[df_1.train==1].index

In [ ]:
# Индексы теста

df_1[df_1.train==0].index

In [ ]:
cat_features_1[:85352].shape

In [ ]:
cat_features_2[85352:, :].shape

In [ ]:
cat_features_1.shape

In [ ]:
def mape(y_true, y_pred):
    '''Функция для определения метрики MAPE'''
    return np.mean(np.abs((y_pred-y_true)/y_true))*100

In [ ]:
# Трейн со стандартизованными числовыми признаками и категориальными признаками c bodyType_1 закодированными по ohe

train_st_oh_1 = np.hstack((ordinal_features, standart_nums, TSVD_features, df_1[bin_cols].values, cat_features_1.toarray()))[:85352]

# Тест со стандартизованными числовыми признаками и категориальными признаками c bodyType_1 закодированными по ohe

test_st_oh_1 = np.hstack((ordinal_features, standart_nums, TSVD_features, df_1[bin_cols].values, cat_features_1.toarray()))[85352:]

In [ ]:
# Создадим простую модель чтобы определить, стоит ли сокращать категориальные переменные или нет

X = train_st_oh_1
y = df_1.price_log.values[:85352]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
test_model_1 = RandomForestRegressor(random_state=RANDOM_SEED)
test_model_1.fit(X_train, y_train)

In [ ]:
rf_train_pred_1 = test_model_1.predict(X_train)
rf_pred_1 = test_model_1.predict(X_test)
print(f"Точность модели Random Forest 1 по метрике MAPE на трейне: {(mape(np.exp(y_train), np.exp(rf_train_pred_1))):0.3f}%")
print(f"Точность модели Random Forest 1 по метрике MAPE на тесте: {(mape(np.exp(y_test), np.exp(rf_pred_1))):0.3f}%")

In [ ]:
# Трейн со стандартизованными числовыми признаками и категориальными признаками c bodyType_2 закодированными по ohe

train_st_oh_2 = np.hstack((ordinal_features, standart_nums, TSVD_features, df_1[bin_cols].values, cat_features_2.toarray()))[:85352]

# Тест со стандартизованными числовыми признаками и категориальными признаками c bodyType_2 закодированными по ohe

test_st_oh_2 = np.hstack((ordinal_features, standart_nums, TSVD_features, df_1[bin_cols].values, cat_features_2.toarray()))[85352:]

In [ ]:
# Создадим простую модель чтобы определить, стоит ли сокращать категориальные переменные или нет

X = train_st_oh_2
y = df_1.price_log.values[:85352]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
test_model_2 = RandomForestRegressor(random_state=RANDOM_SEED)
test_model_2.fit(X_train, y_train)

In [ ]:
rf_train_pred_2 = test_model_2.predict(X_train)
rf_pred_2 = test_model_2.predict(X_test)
print(f"Точность модели Random Forest по метрике MAPE на трейне: {(mape(np.exp(y_train), np.exp(rf_train_pred_2))):0.3f}%")
print(f"Точность модели Random Forest по метрике MAPE на тесте: {(mape(np.exp(y_test), np.exp(rf_pred_2))):0.3f}%")

По итогу будем использовать признак bodyType_1

# Machine Learning

In [ ]:
# Подготовим данные для обучении на основании теста выше

X = train_st_oh_1
y = df_1.price_log.values[:85352]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

Модель для сравнения будем использовать упомянутую выше (RandomForestRegressor без параметров с mape 12.607)

## CatBoost

In [ ]:
# Обучение модели

model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True
         )

In [ ]:
# оцениваем точность

cat_pred_train = model.predict(X_train)
cat_pred = model.predict(X_test)
print(f"Точность модели CatBoost по метрике MAPE на трейне: {(mape(np.exp(y_train), np.exp(cat_pred_train))):0.3f}%")
print(f"Точность модели CatBoost по метрике MAPE на тесте: {(mape(np.exp(y_test), np.exp(cat_pred))):0.3f}%")

## Градиентный бустинг (LightGBM)

In [ ]:
lgbm = lgb.LGBMRegressor(n_estimators=1000, random_state=RANDOM_SEED)
lgbm.fit(X_train, y_train)

In [ ]:
lgbm_pred_train = lgbm.predict(X_train)
lgbm_pred = lgbm.predict(X_test)
print(f"Точность модели LightGBM по метрике MAPE на трейне: {(mape(np.exp(y_train), np.exp(lgbm_pred_train))):0.3f}%")
print(f"Точность модели LightGBM по метрике MAPE на тесте: {(mape(np.exp(y_test), np.exp(lgbm_pred))):0.3f}%")

## ExtraTreesRegressor

In [ ]:
etr = ExtraTreesRegressor(random_state=RANDOM_SEED)
etr.fit(X_train, y_train)

In [ ]:
etr_pred_train = etr.predict(X_train)
etr_pred = etr.predict(X_test)
print(f"Точность модели ExtraTreesRegressor по метрике MAPE на трейне: {(mape(np.exp(y_train), np.exp(etr_pred_train))):0.3f}%")
print(f"Точность модели ExtraTreesRegressor по метрике MAPE на тесте: {(mape(np.exp(y_test), np.exp(etr_pred))):0.3f}%")

На лицо переобучение модели хоть и по тесту это лучший результат. Исправим это на подборе параметров

## Бэггинг

In [ ]:
# в качестве базовой модели будем использовать модель деревьев решений

dt = DecisionTreeRegressor(random_state=RANDOM_SEED)
bct = BaggingRegressor(base_estimator=dt, random_state=RANDOM_SEED)
bct.fit(X_train, y_train)

In [ ]:
bct_pred_train = bct.predict(X_train)
bct_pred = bct.predict(X_test)
print(f"Точность бэггинга с деревьями решений по метрике MAPE на трейне: {(mape(np.exp(y_train), np.exp(bct_pred_train))):0.3f}%")
print(f"Точность бэггинга с деревьями решений по метрике MAPE на тесте: {(mape(np.exp(y_test), np.exp(bct_pred))):0.3f}%")

## Усреднение предсказаний

По итогу тестирования моделей отбираем три с лучшим результатом и усредним их значения расставив веса в качестве коэффициентов в зависимости от метрики (Чем лучше результат, тем больше вес)

In [ ]:
# Найдем среднее среди предсказаний экстремальных деревьев, случайного леса и лгбм с коэффициентами соответственно 0.4, 0.2, 0.4

pred_df = pd.DataFrame({'random_forest_log':rf_pred_1, 'extra_trees_log':etr_pred, 'light_gbm_log': lgbm_pred, 'price_log' : y_test})
pred_df

In [ ]:
final_pred = etr_pred*0.4 + rf_pred_1*0.2 + lgbm_pred*0.4
print(f"Точность по метрике MAPE после усреднения предсказаний: {(mape(np.exp(y_test), np.exp(final_pred))):0.3f}%")

По итогу усреднение результатов модели с распределением коэффициентов в зависимости от точности модели дало лучший результат по метрике. Приступим к подбору параметров для моделей

## Подбор гиперпараметров и построение финальной модели

Закомментировал подбор параметров, так как слишком долго проходит

In [ ]:
### Random Forest ###
#
#rf_grid = {'n_estimators' : [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)],
#           'max_depth' : [3, 5, 7, 10, 15, None],
#           'min_samples_split' : [2, 4, 6]  
#           }
#
#rf = RandomForestRegressor(random_state=RANDOM_SEED)
#rf_randcv = RandomizedSearchCV(estimator = rf, 
#                                param_distributions = rf_grid, 
#                                n_iter = 20, 
#                                cv = 3, 
#                                verbose=2, 
#                                random_state=RANDOM_SEED)
#%time rf_randcv.fit(X_train, y_train)

In [ ]:
### Extratrees ###
#
#etr_grid = {'n_estimators' : [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)],
#           'max_depth' : [3, 5, 7, 10, 15, None],
#           'min_samples_split' : [2, 4, 6],
#           'bootstrap' : [True, False]
#           }
#
#etr = ExtraTreesRegressor(random_state=RANDOM_SEED)
#etr_grid = RandomizedSearchCV(estimator = etr,
#                        param_distributions = etr_grid, 
#                        cv = 3, 
#                        verbose=2)
#
#%time etr_grid.fit(X_train, y_train)

По итогу подбора параметров:
- на каждой из моделей улучшилась метрика;
- избавились от переобучения в ExtraTreesRegressor благодаря бутстрэппингу

In [ ]:
# Конечные данные трейн и тест

X_train = train_st_oh_1
y_train = df_1.price_log.values[:85352]
X_test = test_st_oh_1

In [ ]:
# Random Forest

rf = RandomForestRegressor(n_estimators=300,  random_state=RANDOM_SEED, min_samples_split=4)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

In [ ]:
# ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=300, bootstrap=True, min_samples_split=4, random_state=RANDOM_SEED)
etr.fit(X_train, y_train)

etr_pred = etr.predict(X_test)

In [ ]:
# LightGBM

lgbm = lgb.LGBMRegressor(n_estimators=2500, random_state=RANDOM_SEED, lambda_l2=0.3)
lgbm.fit(X_train, y_train)

lgbm_pred = lgbm.predict(X_test)

Изменим веса наших моделей

In [ ]:
# Найдем среднее среди предсказаний экстремальных деревьев, случайного леса и лгбм с коэффициентами соответственно 0.4, 0.2, 0.4

final_pred = etr_pred*0.3 + rf_pred*0.2 + lgbm_pred*0.5

In [ ]:
# Так как предсказания прологорифмированы найдем значение цены в рублях при помощи экспоненты

submission['price'] = np.exp(final_pred)
submission

In [ ]:
submission.to_csv(f'submission.csv', index=False)

10.64771 результат на каггле, попробуем округлить до сотых

In [ ]:
submission['price'] = np.round(submission['price']/10000,2)*10000

In [ ]:
submission.to_csv(f'submission_1.csv', index=False)
files.download("submission_1.csv")

результат улучшился до 10.64765. На момент парсинга теста (20 сентября 2020 года) по информации из столбца parsingunixtime, курс доллара был 75.0319 RUB. В то же время на момент парсинга трейна (9 ноября 2020 года) курс составлял 77,1875 RUB. Скорректируем наш сабмишн в соответствии с изменением курса (умножим на 1.02)


In [ ]:
submission = pd.read_csv(PATH+'sample_submission.csv')
submission['price'] = np.round((np.exp(final_pred) * 1.02)/10000,2)*10000

In [ ]:
submission.to_csv(f'submission_4.csv', index=False)
files.download("submission_4.csv")

Итоговый результат на Kaggle 10.56356 за 5 сабмишенов и 10 место в лидерборде.

**Спасибо за внимание**